In [1]:
import tensorflow as tf
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import pickle
import time
from xgboost import XGBClassifier
from xgboost import XGBRegressor
import sys
sys.path.append('/work/users/d/d/ddinh/aaco/src')
from load_dataset import load_adni_data
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_curve
import tensorflow as tf


2025-01-07 12:44:04.415740: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-07 12:44:04.792254: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-07 12:44:04.892507: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-07 12:44:05.617770: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-07 12:44:11.484388: W tensorflow/compiler/tf2

## Data

In [2]:
# X_train_digits = np.load('/work/users/d/d/ddinh/aaco/input_data/Synthetic_raw/train_digits.npy')
# X_train_counter = np.load('/work/users/d/d/ddinh/aaco/input_data/Synthetic_raw/train_counter.npy')
# y_train = np.load('/work/users/d/d/ddinh/aaco/input_data/Synthetic_raw/train_labels.npy')
# X_test_digits = np.load('/work/users/d/d/ddinh/aaco/input_data/Synthetic_raw/test_digits.npy')
# X_test_counter = np.load('/work/users/d/d/ddinh/aaco/input_data/Synthetic_raw/test_counter.npy')
# y_test = np.load('/work/users/d/d/ddinh/aaco/input_data/Synthetic_raw/test_labels.npy')

X_train_digits = np.load('/work/users/d/d/ddinh/aaco/input_data/pca_digits/train_digits_images.npy')
X_train_counter = np.load('/work/users/d/d/ddinh/aaco/input_data/pca_digits/train_counter_images.npy')
y_train = np.load('/work/users/d/d/ddinh/aaco/input_data/pca_digits/train_labels.npy')
X_test_digits = np.load('/work/users/d/d/ddinh/aaco/input_data/pca_digits/test_digits_images.npy')
X_test_counter = np.load('/work/users/d/d/ddinh/aaco/input_data/pca_digits/test_counter_images.npy')
y_test = np.load('/work/users/d/d/ddinh/aaco/input_data/pca_digits/test_labels.npy')

y_train = np.eye(np.max(y_train) + 1)[y_train]
y_test = np.eye(np.max(y_test) + 1)[y_test]

X_train = np.concatenate([X_train_digits, X_train_counter], axis=1)
X_test = np.concatenate([X_test_digits, X_test_counter], axis=1)

num_ts = 10
Xval = X_test
Yval = y_test

In [14]:
# load pt file 
import torch 
# path = '/work/users/d/d/ddinh/aaco/results/dataset_adnimlp_imputation_exclude_nan_mask_less_features.pt'
# path = '/work/users/d/d/ddinh/aaco/results/dataset_synthetic_raw_siamese_0.03.pt'
# path = '/work/users/d/d/ddinh/aaco/results/dataset_synthetic_raw_xgb_0.025.pt'
path = '/work/users/d/d/ddinh/aaco/results/dataset_synthetic_pca_xgb_0.03.pt'
data = torch.load(path)
x = data['X']
y = data['y']
mask = data['mask']
action = data['Action']

In [15]:
merge_action = []
count = 0
for i in range(action.shape[0]):
    if action[i, 0] == 1:
        merge_action.append(action[i])
        count += 1
    else:
        merge_action[count - 1] += action[i]
merge_action = np.array(merge_action)
merge_action = merge_action[:,:-1]

In [16]:
import tensorflow as tf
# model = tf.keras.models.load_model('/work/users/d/d/ddinh/aaco/models/mlp.keras')
model = XGBClassifier()
# model.load_model('/work/users/d/d/ddinh/aaco/models/synthetic_raw.model')
model.load_model('/work/users/d/d/ddinh/aaco/models/synthetic_pca.model')

In [17]:
merge_action.shape

(10000, 20)

In [18]:
current = merge_action.shape[0]
merge_action = merge_action[:current]
Xval = Xval[:current]
Yval = Yval[:current]


In [19]:
# d = 24
# B = np.concatenate(
# [np.sum(np.random.permutation(np.eye(d))[:, :np.random.randint(6)], 1, keepdims=True) for _ in range(merge_action.shape[0])],
# 1)
# B = np.float32(B.T)
# # B = np.expand_dims(B, axis=-1)

# merge_action = B
# np.sum(merge_action, 1).mean()
# merge_action = np.ones_like(merge_action)

In [20]:
x_input = (Xval*merge_action[:,:,None]).reshape(Xval.shape[0], -1)
Xvalmasked = np.concatenate((x_input, merge_action), 1)
val_preds = model.predict_proba(Xvalmasked) 
print(np.mean(np.round(val_preds)==Yval))

0.9026818181818181


In [21]:
np.sum(merge_action[:,:10], 1).mean() / 10, np.sum(merge_action[:,10:], 1).mean() / 10

(0.712559986114502, 0.6570899963378907)

In [11]:
# raw 
0.02: 1800
0.9496969696969697
(0.6128333568572998, 0.530388879776001)


0.025: 2400 samples
0.9451515151515152
(0.5213749885559082, 0.4473750114440918)


SyntaxError: illegal target for annotation (1846374165.py, line 2)

In [48]:
# siamese: 
0.02: 1800 samples
0.949040404040404    
(0.6426111221313476, 0.5537777900695801)

0.025: 2400 samples
0.9410227272727273
(0.4872499942779541, 0.42216668128967283)

SyntaxError: illegal target for annotation (2052525778.py, line 2)